# Import biblioteka

In [1]:
#import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import math
#import pylab 
#import scipy.stats as stats

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import  StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

import datetime

# Učitavanje i analiza podataka

In [2]:
X_test= pd.read_csv('X_test.txt', sep='\s+')
y_test= pd.read_csv('y_test.txt', sep='\s+')
X_train= pd.read_csv('X_train.txt', sep='\s+')
y_train= pd.read_csv('y_train.txt', sep='\s+')

In [3]:
podaci_test = pd.concat([X_test, y_test], axis=1)
podaci_train = pd.concat([X_train, y_train], axis=1)

In [4]:
podaci = pd.concat([podaci_train, podaci_test], axis=0)

In [5]:
def matrica_predv(conf_matrica):
    cm_df = pd.DataFrame(conf_matrica, index=[f"Стварна вриједност {i+1}" for i in range(6)], columns=[f"Предикција {i+1}" for i in range(6)])

    # Prikaz tabele
    return cm_df

In [6]:
def klas_izv (y_skup,X_skup_predikcija, skup):
    izvjestaj=metrics.classification_report(y_skup, X_skup_predikcija, output_dict=True)
     
    izvjestaj_df=pd.DataFrame(izvjestaj).transpose()
    izvjestaj_df['support'] = izvjestaj_df['support'].round().astype(int)
    
    izvjestaj_df = izvjestaj_df.rename(columns={
            'precision': 'Прецизност',
            'recall': 'Одзив',
            'f1-score': 'f1-score',
            'support': 'Број узорака у класи'  # Ovaj naziv se koristi za broj uzoraka u svakoj klasi
        })

    izvjestaj_df = izvjestaj_df.rename(index={
            '1': 'Пјешачење',
            '2': 'Пењање уз степенице',
            '3': 'Силазак низ степенице',
            '4': 'Сједење',
            '5': 'Стајање',
            '6': 'Лежање',
            'accuracy': 'Тачност',
            'macro avg': 'Макро просјек',  #prosečne vrijednosti metrika za sve klase, bez ponderisanja u zavisnosti od broja uzoraka u svakoj klasi.
            'weighted avg': 'Пондерисани просјек'
        })
 
   
        
    return  izvjestaj_df

# Standardizacija 

In [7]:
#Standardizacija trening skupa

# Za kolonu Class ne radi se standardizacija

# Inicijalizacija StandardScaler
scaler = StandardScaler()

# Primjena standardizacije
st_X_train = scaler.fit_transform(X_train)

# Prevođenje podataka u dataframe
st_X_train = pd.DataFrame(st_X_train, columns=X_train.columns)

st_X_train

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.200642,-0.063683,-0.419628,-0.868814,-0.939441,-0.737529,-0.859817,-0.939019,-0.766437,-0.856036,...,-0.795359,0.025960,-0.276399,-0.360603,0.062940,-0.778427,-0.026080,-0.687219,0.407946,-0.007568
1,0.055948,0.031486,-0.253908,-0.875426,-0.923902,-0.849304,-0.868531,-0.921998,-0.848928,-0.871359,...,0.130614,-0.897357,-0.767990,0.133011,-0.021461,-1.218805,1.484470,-0.694138,0.409117,0.007875
2,0.073515,-0.043416,-0.076295,-0.869039,-0.907760,-0.893785,-0.863137,-0.898854,-0.896701,-0.863323,...,1.152336,-0.260878,-0.438316,-0.377840,0.391976,0.151207,1.704201,-0.702239,0.410288,0.026502
3,0.066696,-0.208422,-0.249712,-0.870626,-0.940022,-0.921805,-0.864503,-0.938124,-0.925279,-0.863323,...,1.112769,0.591045,0.463155,-0.135025,-0.033637,1.037851,-1.003019,-0.701684,0.414650,0.031714
4,0.030469,0.027587,-0.109848,-0.875188,-0.934878,-0.921343,-0.867384,-0.931789,-0.928028,-0.870260,...,-0.149577,-0.138515,-0.240313,0.340406,0.268486,1.125918,-1.276282,-0.700152,0.425463,0.045225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.358361,-0.967904,-1.273005,0.913859,1.095963,1.628612,0.821169,1.174137,1.609686,1.247727,...,-0.778758,-0.876738,-0.829197,-0.591277,1.846034,0.325923,-0.878008,-0.590763,0.605162,0.380984
7348,-0.009044,0.243731,-0.676787,0.824887,1.026150,1.586100,0.726245,1.023755,1.658407,1.076279,...,0.159241,-0.260847,-0.180290,0.166951,1.948561,-1.459501,0.849894,-0.551598,0.652468,0.381820
7349,-0.015668,0.016781,1.132221,0.862975,0.810002,2.100249,0.768781,0.880813,2.266963,0.941403,...,0.277877,1.034784,1.044548,0.131018,-0.599877,1.406760,1.479571,-0.565848,0.640597,0.348709
7350,0.215866,-0.028123,-0.867710,0.860922,0.794902,2.086778,0.754697,0.944019,2.235301,1.047375,...,-0.443978,1.155541,0.913569,-0.326769,1.558312,1.525574,-1.220854,-0.577668,0.631478,0.293276


In [8]:
#Standardizacija testnog skupa

# Za kolonu Class ne radi se standardizacija

# Inicijalizacija StandardScaler
scaler = StandardScaler()

# Primjena standardizacije
st_X_test = scaler.fit_transform(X_test)

# Prevođenje podataka u dataframe
st_X_test = pd.DataFrame(st_X_test, columns=X_test.columns)


st_X_test

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,-0.277708,-0.210631,2.193076,-0.787267,-0.833212,-0.093444,-0.808091,-0.839013,-0.104178,-0.824746,...,-0.253663,-0.166391,-0.344765,0.003566,0.357573,-1.363900,0.575487,-0.404790,0.622723,-0.038348
1,0.198660,0.182591,-0.250264,-0.876984,-0.929061,-0.858050,-0.897145,-0.928949,-0.863269,-0.824746,...,-2.300750,0.491021,0.012257,-0.264092,0.030787,-0.747232,1.871192,-0.361739,0.636731,-0.145709
2,0.024587,-0.318055,-0.228485,-0.921598,-0.934054,-0.907107,-0.916891,-0.933813,-0.912639,-0.910982,...,-0.291563,0.274821,-0.134989,-0.119671,0.446072,0.037919,0.323938,-0.370932,0.632845,-0.126855
3,-0.061057,-0.573037,-0.213709,-0.923837,-0.940817,-0.919082,-0.919152,-0.941599,-0.927776,-0.909740,...,-0.058625,-0.210678,-0.441714,-0.066443,0.338512,0.472695,0.625874,-0.363434,0.645276,-0.117585
4,0.013823,-0.387881,-0.494639,-0.921679,-0.929036,-0.949979,-0.916133,-0.923846,-0.951396,-0.909740,...,-1.090781,-0.810527,-0.796939,-0.022276,-0.098529,1.097348,-0.201999,-0.350256,0.665657,-0.104118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.597072,-1.380212,0.217067,0.789690,0.744135,1.154623,0.740507,0.779257,1.132204,1.235951,...,-0.241427,-0.311121,-0.488935,-1.019623,0.769650,1.330760,-1.359840,-0.270682,0.616018,0.967186
2943,1.476032,-0.829444,0.057817,0.747216,1.085580,1.206230,0.695181,1.026724,1.025466,1.235951,...,-0.122855,-0.135013,-0.326438,-2.207635,-0.846487,-1.098551,0.678029,-0.277456,0.612784,0.957360
2944,1.254458,1.862441,-0.173180,0.688432,0.943345,1.057851,0.657591,0.811598,0.810778,0.896354,...,-0.850662,0.500453,0.423089,-0.555875,0.190504,1.034263,0.759011,-0.277802,0.615561,0.952271
2945,-0.601088,1.411385,0.278129,0.704246,0.563664,1.175343,0.646262,0.506579,0.971675,0.896354,...,-0.765017,0.227458,-0.061034,1.307070,-1.849401,1.400679,0.017755,-0.286369,0.585654,0.978695


# Podjela podataka za primjenu modela

In [9]:
X_train=st_X_train
X_test=st_X_test

In [10]:
#Podaci su već podijeljeni u inicijalnom učitavanju - takav je dataset
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(7352, 561) (7352, 1) (2947, 561) (2947, 1)


In [11]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# SVD
TruncatedSVD

In [12]:
def gridSearch (X_train, y_train, broj_komponenti):
    param_grid = {'penalty':['l2'],
               'C' : [0.01,0.05,0.1], 
               'solver': ['lbfgs','newton-cg','saga'],
                'multi_class': ['multinomial'],
               'max_iter'  : [2000,2500,2800],
               'class_weight':['balanced']
              }
                  
    log_model = LogisticRegression()
    grid_search = GridSearchCV(log_model,param_grid = param_grid, cv = 5, verbose=2)

    # Treniranje modela sa GridSearchCV
    grid_search.fit(X_train, y_train) 

    rezultati_df = pd.DataFrame(grid_search.cv_results_)

    # Ispis najboljih parametara
    print(f"Најбољи параметри: {grid_search.best_params_}")  ##otkomentarisati poslije
    return grid_search.best_estimator_  

-----***Primjena SVD i logističke regresije***-----

In [13]:
def log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti):
        t1 = datetime.datetime.now()
        print('Početak izvršenja: ',t1)
        
        #Provjerićemo za dobijeni broj komponenti i detaljnije
        svd = TruncatedSVD(n_components=broj_komponenti)  #454,539,741
        X_train_svd = svd.fit_transform(X_train)
        X_test_svd = svd.transform(X_test)
                
        print('Матрица добијена примјеном SVD на тестне податке \n', X_test_svd.round(6))
        print('Mатрица десно сингуларних вектора V^T \n',svd.components_.round(6))
        print("Сингуларне вриједности:\n", svd.singular_values_.round(6))
        
        # treniranje modela
        t2= datetime.datetime.now()
        print('Početak gridsearch: ',t2)
        najbolji = gridSearch(X_train_svd, y_train,broj_komponenti)
        t3= datetime.datetime.now()
        print('Kraj gridsearch: ',t3)
        print('Trajanje gridsearch u sekundama ',(t3-t2).total_seconds())

        #tačnost modela na trening podacima
        X_train_svd_predict = najbolji.predict (X_train_svd)
        tacnost_training_svd_data = metrics.accuracy_score(y_train,X_train_svd_predict)
        conf_matrica_svd_training = metrics.confusion_matrix(y_train, X_train_svd_predict).round(6)
        print('Тачност процијењених тренинг података:',tacnost_training_svd_data.round(6))
        
        display(klas_izv(y_train, X_train_svd_predict, 'тренинг'))
        #display(matrica_predv(conf_matrica_svd_training))
        
        #tačnost modela na testnim podacima
        X_test_svd_predict = najbolji.predict (X_test_svd)
        tacnost_test_svd_data = metrics.accuracy_score(y_test,X_test_svd_predict)
        conf_matrica_svd = metrics.confusion_matrix(y_test, X_test_svd_predict).round(6)
        print('Тачност процијењених тестних података:',tacnost_test_svd_data.round(6))
        
        #display(matrica_predv(conf_matrica_svd))
        #klasifikacioni izvjestaj
        display(klas_izv(y_test, X_test_svd_predict, 'тест'))
        
        #objasnjena varijansa svake od glavnih komponenti
         #objasnjena varijansa svake od glavnih komponenti
        print('Објашњена варијанса сваке од главних компоненти:',svd.explained_variance_.round(6)) 

        #postotak objasnjene varijanse svake od glavnih komponenti
        print('Постотак објашњене варијансе сваке од главних компоненти:',svd.explained_variance_ratio_.round(6) )
        print('Укупни проценат кумулативне објашњене варијансе ', sum(svd.explained_variance_ratio_).round(6)*100)
        
        t4 = datetime.datetime.now()
        print('Kraj izvršenja: ',t4)
        print('Trajanje izvršenja u sekundama ',(t4-t1).total_seconds())

In [14]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=179)

Početak izvršenja:  2025-04-20 22:28:48.817695
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119  -1.861169   2.22811  ...  -0.417461   0.544333  -0.387814]
 [-14.113888  -1.077893  -0.396704 ...   0.160623   0.423868   0.019668]
 [-15.324244   2.365216  -2.253297 ...  -0.072039   0.131204  -0.261318]
 ...
 [ 13.045278  -2.908067   3.207745 ...  -0.215186   0.465516   0.174703]
 [  9.080038  -3.970685   1.926921 ...  -0.215259   0.389248  -0.113492]
 [ 10.361516  -3.453699   2.291647 ...   0.098019   0.256963   0.576959]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.80000e-05 -2.47300e-03 -1.52700e-03 ... -2.31000e-02  2.97190e-02
   2.66220e-02]
 [ 1.56380e-02 -8.00000e-05 -4.05200e-03 ...  1.74760e-02 -3.52130e-02
  -1.72780e-02]
 [-3.24820e-02  2.69500e-03  1.01670e-02 ...  1.07508e-01 -8.00530e-02
  -6.44240e-02]
 ...
 [ 2.75600e-02 -2.55150e-02 -2.82890e-02 ... -1.16790e-02 -2.50060e-02
  -3.69600e-03]
 [-5.34820e-02  1.68760e-02  5.61910e-02 ...  2.83340e-02  4.7501

[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.2s
[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.4s
[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   8.2s
[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   8.2s
[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   4.8s
[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   8.1s
[CV] END C=0.01, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   8.5s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.

[CV] END C=0.1, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=  36.6s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.9s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   2.1s
[

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,1.000000,1.000000,1.000000,1226
Пењање уз степенице,1.000000,1.000000,1.000000,1073
Силазак низ степенице,1.000000,1.000000,1.000000,986
Сједење,0.961059,0.959565,0.960311,1286
Стајање,0.962209,0.963610,0.962909,1374
Лежање,1.000000,1.000000,1.000000,1407
Тачност,0.986126,0.986126,0.986126,1
Макро просјек,0.987211,0.987196,0.987203,7352
Пондерисани просјек,0.986126,0.986126,0.986126,7352


Тачност процијењених тестних података: 0.947404


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.969574,0.963710,0.966633,496
Пењање уз степенице,0.950000,0.968153,0.958991,471
Силазак низ степенице,0.964029,0.957143,0.960573,420
Сједење,0.907598,0.900204,0.903885,491
Стајање,0.896869,0.915414,0.906047,532
Лежање,1.000000,0.981378,0.990602,537
Тачност,0.947404,0.947404,0.947404,1
Макро просјек,0.948012,0.947667,0.947788,2947
Пондерисани просјек,0.947749,0.947404,0.947524,2947


Објашњена варијанса сваке од главних компоненти: [2.84882377e+02 3.69176160e+01 1.57441100e+01 1.40471750e+01
 1.05932790e+01 9.67167600e+00 7.69137400e+00 6.72682600e+00
 5.58677200e+00 5.41413600e+00 4.82489000e+00 4.48957600e+00
 4.28602000e+00 3.62029500e+00 3.54734000e+00 3.36444100e+00
 3.29166900e+00 3.22803500e+00 3.18603700e+00 2.95812900e+00
 2.80703600e+00 2.73997100e+00 2.68273400e+00 2.62863800e+00
 2.52051300e+00 2.36196300e+00 2.34486000e+00 2.27525000e+00
 2.17962100e+00 2.17117100e+00 2.05302900e+00 1.98904400e+00
 1.95195800e+00 1.89162000e+00 1.84948800e+00 1.84128600e+00
 1.79817000e+00 1.65840000e+00 1.60851300e+00 1.59926800e+00
 1.50777900e+00 1.49012700e+00 1.47770800e+00 1.45215200e+00
 1.39719800e+00 1.38541200e+00 1.35021400e+00 1.32460600e+00
 1.29186400e+00 1.27665200e+00 1.23512400e+00 1.19755000e+00
 1.16522200e+00 1.13600600e+00 1.12076300e+00 1.10938100e+00
 1.08987500e+00 1.06723000e+00 1.05504400e+00 1.04467200e+00
 1.01802700e+00 9.95107000e-01 9.878

In [15]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=102)

Početak izvršenja:  2025-04-20 22:46:51.807084
Матрица добијена примјеном SVD на тестне податке 
 [[-1.0945119e+01 -1.8611690e+00  2.2281100e+00 ...  4.2329800e-01
  -7.8152000e-01 -1.5681300e-01]
 [-1.4113888e+01 -1.0778930e+00 -3.9670400e-01 ... -5.5483100e-01
  -1.7334000e-01  5.6899000e-02]
 [-1.5324244e+01  2.3652160e+00 -2.2532970e+00 ...  1.3807830e+00
  -5.3586700e-01  2.5254600e-01]
 ...
 [ 1.3045278e+01 -2.9080670e+00  3.2077450e+00 ...  7.1292200e-01
  -1.0678240e+00 -3.9997300e-01]
 [ 9.0800380e+00 -3.9706850e+00  1.9269210e+00 ...  1.2645600e-01
  -8.7905000e-02  1.0611260e+00]
 [ 1.0361516e+01 -3.4536990e+00  2.2916470e+00 ... -5.1912200e-01
  -1.3800100e-01  1.6270000e-03]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.80000e-05 -2.47300e-03 -1.52700e-03 ... -2.31000e-02  2.97190e-02
   2.66220e-02]
 [ 1.56380e-02 -8.00000e-05 -4.05200e-03 ...  1.74760e-02 -3.52130e-02
  -1.72780e-02]
 [-3.24820e-02  2.69500e-03  1.01670e-02 ...  1.07508e-01 -8.00530e-02
  -6.44240e-02]
 

[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.9s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.7s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.7s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.4s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.5s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total t

[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.4s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.6s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.4s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  18.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  16.6s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   7.8s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  16

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,1.000000,0.999184,0.999592,1226
Пењање уз степенице,0.999069,1.000000,0.999534,1073
Силазак низ степенице,1.000000,1.000000,1.000000,986
Сједење,0.942813,0.948678,0.945736,1286
Стајање,0.951684,0.946143,0.948905,1374
Лежање,1.000000,1.000000,1.000000,1407
Тачност,0.980822,0.980822,0.980822,1
Макро просјек,0.982261,0.982334,0.982295,7352
Пондерисани просјек,0.980831,0.980822,0.980823,7352


Тачност процијењених тестних података: 0.938242


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.959514,0.955645,0.957576,496
Пењање уз степенице,0.935897,0.929936,0.932907,471
Силазак низ степенице,0.927570,0.945238,0.936321,420
Сједење,0.904762,0.890020,0.897331,491
Стајање,0.898336,0.913534,0.905871,532
Лежање,1.000000,0.992551,0.996262,537
Тачност,0.938242,0.938242,0.938242,1
Макро просјек,0.937680,0.937821,0.937711,2947
Пондерисани просјек,0.938398,0.938242,0.938282,2947


Објашњена варијанса сваке од главних компоненти: [284.882377  36.917616  15.74411   14.047175  10.593279   9.671676
   7.691374   6.726826   5.586772   5.414136   4.82489    4.489576
   4.28602    3.620295   3.54734    3.364441   3.291669   3.228035
   3.186037   2.958129   2.807036   2.739971   2.682734   2.628638
   2.520513   2.361963   2.34486    2.27525    2.179621   2.171171
   2.053029   1.989044   1.951958   1.89162    1.849488   1.841286
   1.79817    1.6584     1.608513   1.599268   1.507779   1.490127
   1.477707   1.452151   1.397197   1.385412   1.350214   1.324604
   1.291861   1.276651   1.235119   1.197542   1.165218   1.136001
   1.120761   1.109378   1.089869   1.067219   1.055036   1.044659
   1.018018   0.995027   0.987814   0.972053   0.957834   0.945532
   0.929082   0.914891   0.888669   0.883508   0.85838    0.853777
   0.830946   0.819287   0.811237   0.80052    0.777631   0.759583
   0.753932   0.745744   0.732901   0.730119   0.718905   0.700482
   0.689608  

In [16]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=76)

Početak izvršenja:  2025-04-20 22:56:40.389723
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119  -1.861169   2.22811  ...   0.343559  -0.663806   0.180978]
 [-14.113888  -1.077893  -0.396704 ...  -0.187157   0.654113   0.439895]
 [-15.324244   2.365216  -2.253297 ...  -0.031872   0.329686   0.28229 ]
 ...
 [ 13.045278  -2.908067   3.207745 ...   2.048948  -0.602753  -1.781093]
 [  9.080038  -3.970685   1.926921 ...  -0.220313  -0.581025  -1.298701]
 [ 10.361516  -3.453699   2.291647 ...   1.875604  -1.732699   0.176519]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.80000e-05 -2.47300e-03 -1.52700e-03 ... -2.31000e-02  2.97190e-02
   2.66220e-02]
 [ 1.56380e-02 -8.00000e-05 -4.05200e-03 ...  1.74760e-02 -3.52130e-02
  -1.72780e-02]
 [-3.24820e-02  2.69500e-03  1.01670e-02 ...  1.07508e-01 -8.00530e-02
  -6.44240e-02]
 ...
 [ 6.42980e-02  4.35570e-02 -1.16839e-01 ...  1.39310e-02  2.94390e-02
  -8.29600e-03]
 [ 5.98670e-02  8.07400e-02  6.03600e-03 ... -1.33010e-02 -1.5297

[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.4s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.3s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.3s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.3s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.6s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; 

[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.8s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  13.0s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  12.4s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   5.4s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  12.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  12.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.9s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.1s
[CV] END 

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.999182,0.996737,0.997958,1226
Пењање уз степенице,0.997207,0.998136,0.997671,1073
Силазак низ степенице,0.996964,0.998986,0.997974,986
Сједење,0.917852,0.903577,0.910658,1286
Стајање,0.912356,0.924309,0.918294,1374
Лежање,0.998581,1.000000,0.999290,1407
Тачност,0.968036,0.968036,0.968036,1
Макро просјек,0.970357,0.970291,0.970307,7352
Пондерисани просјек,0.968028,0.968036,0.968015,7352


Тачност процијењених тестних података: 0.929759


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.960251,0.925403,0.942505,496
Пењање уз степенице,0.910973,0.934183,0.922432,471
Силазак низ степенице,0.918224,0.935714,0.926887,420
Сједење,0.869396,0.908350,0.888446,491
Стајање,0.918288,0.887218,0.902486,532
Лежање,0.998117,0.986965,0.992509,537
Тачност,0.929759,0.929759,0.929759,1
Макро просјек,0.929208,0.929639,0.929211,2947
Пондерисани просјек,0.930573,0.929759,0.929952,2947


Објашњена варијанса сваке од главних компоненти: [284.882377  36.917616  15.74411   14.047175  10.593279   9.671676
   7.691374   6.726826   5.586772   5.414136   4.82489    4.489576
   4.28602    3.620295   3.54734    3.364441   3.291669   3.228035
   3.186037   2.958129   2.807036   2.739971   2.682734   2.628638
   2.520512   2.361962   2.34486    2.275249   2.179619   2.17117
   2.053028   1.989041   1.951954   1.891617   1.849483   1.841284
   1.798165   1.658352   1.608504   1.59918    1.507749   1.490076
   1.477542   1.452115   1.397008   1.385259   1.350139   1.324492
   1.291642   1.276586   1.234581   1.196961   1.164548   1.134541
   1.119988   1.10894    1.086605   1.066255   1.053781   1.042632
   1.012561   0.993506   0.983232   0.970114   0.955573   0.939481
   0.925221   0.905479   0.88577    0.871798   0.844238   0.840552
   0.816795   0.801443   0.790532   0.762103]
Постотак објашњене варијансе сваке од главних компоненти: [0.507812 0.065807 0.028064 0.02504  0.01888

In [17]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=63)

Početak izvršenja:  2025-04-20 23:04:38.357767
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119  -1.861169   2.22811  ...   1.448974   0.699088   0.659612]
 [-14.113888  -1.077893  -0.396704 ...  -1.484786   1.273525   0.138263]
 [-15.324244   2.365216  -2.253297 ...  -0.751834   0.770638   0.510015]
 ...
 [ 13.045278  -2.908067   3.207745 ...   2.720018   2.802442   1.152158]
 [  9.080038  -3.970685   1.926921 ...   0.095622  -0.66033    2.193109]
 [ 10.361516  -3.453699   2.291647 ...  -0.780941   0.68706    1.880764]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.80000e-05 -2.47300e-03 -1.52700e-03 ... -2.31000e-02  2.97190e-02
   2.66220e-02]
 [ 1.56380e-02 -8.00000e-05 -4.05200e-03 ...  1.74760e-02 -3.52130e-02
  -1.72780e-02]
 [-3.24820e-02  2.69500e-03  1.01670e-02 ...  1.07508e-01 -8.00530e-02
  -6.44240e-02]
 ...
 [ 1.26350e-01  3.47856e-01  1.27009e-01 ...  1.08660e-02 -3.54200e-03
   2.96930e-02]
 [-3.26500e-02  1.48710e-02 -8.24380e-02 ... -1.77270e-02  3.5154

[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   8.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   7.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; to

[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   9.9s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   5.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  10.4s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=  11.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.2s
[CV] END C=0

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.995918,0.995106,0.995512,1226
Пењање уз степенице,0.996269,0.995340,0.995804,1073
Силазак низ степенице,0.995951,0.997972,0.996960,986
Сједење,0.910954,0.898911,0.904892,1286
Стајање,0.907194,0.917758,0.912446,1374
Лежање,0.999290,1.000000,0.999645,1407
Тачност,0.965180,0.965180,0.965180,1
Макро просјек,0.967596,0.967515,0.967543,7352
Пондерисани просјек,0.965176,0.965180,0.965165,7352


Тачност процијењених тестних података: 0.923651


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.949686,0.913306,0.931141,496
Пењање уз степенице,0.907757,0.919321,0.913502,471
Силазак низ степенице,0.905747,0.938095,0.921637,420
Сједење,0.870775,0.892057,0.881288,491
Стајање,0.904943,0.894737,0.899811,532
Лежање,0.998110,0.983240,0.990619,537
Тачност,0.923651,0.923651,0.923651,1
Макро просјек,0.922836,0.923459,0.923000,2947
Пондерисани просјек,0.924322,0.923651,0.923844,2947


Објашњена варијанса сваке од главних компоненти: [284.882377  36.917616  15.74411   14.047175  10.593279   9.671676
   7.691374   6.726826   5.586772   5.414136   4.82489    4.489576
   4.28602    3.620295   3.54734    3.364441   3.291669   3.228035
   3.186037   2.958129   2.807034   2.73997    2.682732   2.628637
   2.520508   2.361958   2.344849   2.275242   2.179612   2.171161
   2.05302    1.98903    1.951904   1.891581   1.849434   1.841251
   1.798091   1.658209   1.608131   1.598954   1.507279   1.487431
   1.47727    1.450649   1.39623    1.384316   1.345021   1.32086
   1.290736   1.275475   1.231906   1.194882   1.161421   1.130291
   1.111198   1.104526   1.08179    1.052159   1.049316   1.018037
   0.996896   0.964208   0.953081]
Постотак објашњене варијансе сваке од главних компоненти: [0.507812 0.065807 0.028064 0.02504  0.018883 0.01724  0.01371  0.011991
 0.009959 0.009651 0.008601 0.008003 0.00764  0.006453 0.006323 0.005997
 0.005868 0.005754 0.005679 0.005273 0.0050

In [18]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=61)

Početak izvršenja:  2025-04-20 23:11:31.174496
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119  -1.861169   2.22811  ...   1.735936   1.352338  -0.462438]
 [-14.113888  -1.077893  -0.396704 ...   0.276848   0.345968  -1.14333 ]
 [-15.324244   2.365216  -2.253297 ...   1.290442  -0.337997  -0.160634]
 ...
 [ 13.045278  -2.908067   3.207745 ...  -2.047301   0.16352    0.890538]
 [  9.080038  -3.970685   1.926921 ...   1.026435  -0.398491   1.344048]
 [ 10.361516  -3.453699   2.291647 ...  -2.666557   0.689258   1.877829]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.80000e-05 -2.47300e-03 -1.52700e-03 ... -2.31000e-02  2.97190e-02
   2.66220e-02]
 [ 1.56380e-02 -8.00000e-05 -4.05200e-03 ...  1.74760e-02 -3.52130e-02
  -1.72780e-02]
 [-3.24820e-02  2.69500e-03  1.01670e-02 ...  1.07508e-01 -8.00530e-02
  -6.44240e-02]
 ...
 [ 6.40960e-02  3.43008e-01  5.74350e-02 ... -6.20000e-04  4.29460e-02
  -6.32980e-02]
 [-7.59000e-02 -2.32419e-01 -3.69910e-02 ...  1.64700e-02  5.7102

[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.8s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.9s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   1.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   7.7s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   7.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; to

[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   9.0s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   4.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   9.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=saga; total time=   9.6s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   2.0s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   1.8s
[CV] END C=0

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.995110,0.995922,0.995516,1226
Пењање уз степенице,0.995336,0.994408,0.994872,1073
Силазак низ степенице,0.996957,0.996957,0.996957,986
Сједење,0.911742,0.899689,0.905675,1286
Стајање,0.908567,0.918486,0.913500,1374
Лежање,0.998581,1.000000,0.999290,1407
Тачност,0.965316,0.965316,0.965316,1
Макро просјек,0.967715,0.967577,0.967635,7352
Пондерисани просјек,0.965298,0.965316,0.965295,7352


Тачност процијењених тестних података: 0.922294


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.951983,0.919355,0.935385,496
Пењање уз степенице,0.898947,0.906582,0.902748,471
Силазак низ степенице,0.905747,0.938095,0.921637,420
Сједење,0.849906,0.922607,0.884766,491
Стајање,0.929435,0.866541,0.896887,532
Лежање,0.996219,0.981378,0.988743,537
Тачност,0.922294,0.922294,0.922294,1
Макро просјек,0.922040,0.922426,0.921694,2947
Пондерисани просјек,0.923901,0.922294,0.922549,2947


Објашњена варијанса сваке од главних компоненти: [284.882377  36.917616  15.74411   14.047175  10.593279   9.671676
   7.691374   6.726826   5.586772   5.414136   4.82489    4.489576
   4.28602    3.620295   3.54734    3.364441   3.291669   3.228035
   3.186037   2.958128   2.807035   2.739969   2.682733   2.628637
   2.520509   2.361945   2.34484    2.275238   2.179592   2.171136
   2.053003   1.989008   1.951769   1.891547   1.849303   1.841051
   1.798108   1.658065   1.608224   1.599128   1.507321   1.489061
   1.476865   1.451699   1.393244   1.383606   1.347418   1.317511
   1.288363   1.270264   1.233362   1.190302   1.159334   1.129796
   1.111944   1.102198   1.070986   1.038002   1.033513   1.009787
   0.986846]
Постотак објашњене варијансе сваке од главних компоненти: [0.507812 0.065807 0.028064 0.02504  0.018883 0.01724  0.01371  0.011991
 0.009959 0.009651 0.008601 0.008003 0.00764  0.006453 0.006323 0.005997
 0.005868 0.005754 0.005679 0.005273 0.005004 0.004884 0.004782 

In [19]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=3)

Početak izvršenja:  2025-04-20 23:17:54.904851
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119  -1.86117    2.228302]
 [-14.113888  -1.077891  -0.397037]
 [-15.324244   2.365217  -2.253308]
 ...
 [ 13.045278  -2.908065   3.207765]
 [  9.080038  -3.970683   1.926883]
 [ 10.361516  -3.453697   2.291847]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.80000e-05 -2.47300e-03 -1.52700e-03 ... -2.31000e-02  2.97190e-02
   2.66220e-02]
 [ 1.56380e-02 -8.00000e-05 -4.05200e-03 ...  1.74760e-02 -3.52130e-02
  -1.72780e-02]
 [-3.24820e-02  2.69800e-03  1.01740e-02 ...  1.07503e-01 -8.00450e-02
  -6.44320e-02]]
Сингуларне вриједности:
 [1447.223284  520.978229  340.221536]
Početak gridsearch:  2025-04-20 23:17:55.133236
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.4s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, 

[CV] END C=0.05, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.9s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.7s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time= 

[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.3s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.7s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   2.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.785654,0.759380,0.772294,1226
Пењање уз степенице,0.754200,0.794967,0.774047,1073
Силазак низ степенице,0.662898,0.654158,0.658499,986
Сједење,0.499188,0.478227,0.488483,1286
Стајање,0.666905,0.677584,0.672202,1374
Лежање,0.827178,0.843639,0.835327,1407
Тачност,0.702122,0.702122,0.702122,1
Макро просјек,0.699337,0.701326,0.700142,7352
Пондерисани просјек,0.700246,0.702122,0.701002,7352


Тачност процијењених тестних података: 0.688497


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.685714,0.822581,0.747938,496
Пењање уз степенице,0.861407,0.857749,0.859574,471
Силазак низ степенице,0.668731,0.514286,0.581427,420
Сједење,0.458333,0.291242,0.356164,491
Стајање,0.582524,0.902256,0.707965,532
Лежање,0.891509,0.703911,0.786681,537
Тачност,0.688497,0.688497,0.688497,1
Макро просјек,0.691370,0.682004,0.673291,2947
Пондерисани просјек,0.692361,0.688497,0.676619,2947


Објашњена варијанса сваке од главних компоненти: [284.882377  36.917616  15.74411 ]
Постотак објашњене варијансе сваке од главних компоненти: [0.507812 0.065807 0.028064]
Укупни проценат кумулативне објашњене варијансе  60.168299999999995
Kraj izvršenja:  2025-04-20 23:20:07.698210
Trajanje izvršenja u sekundama  132.793359


In [20]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=2)

Početak izvršenja:  2025-04-20 23:20:07.719157
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119  -1.861167]
 [-14.113888  -1.077888]
 [-15.324244   2.365216]
 ...
 [ 13.045278  -2.908058]
 [  9.080038  -3.970676]
 [ 10.361516  -3.453691]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.8000e-05 -2.4730e-03 -1.5270e-03 ... -2.3100e-02  2.9719e-02
   2.6622e-02]
 [ 1.5638e-02 -8.0000e-05 -4.0520e-03 ...  1.7476e-02 -3.5213e-02
  -1.7278e-02]]
Сингуларне вриједности:
 [1447.223284  520.978229]
Početak gridsearch:  2025-04-20 23:20:07.952534
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.4s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.2s
[CV] EN

[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.3s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.2s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=newton-cg; tot

[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.2s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.5s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.1s
[CV] END C=0.1, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=saga; total time=   1.1s
Најбољи параметри: {'C': 0.05, 'class_weight': 'balanced', 'max_iter': 2000, 'multi_class': 'multinomial', 'penalty': 'l2', 'solv

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.778798,0.761011,0.769802,1226
Пењање уз степенице,0.727897,0.790308,0.757819,1073
Силазак низ степенице,0.646996,0.611562,0.628780,986
Сједење,0.285930,0.146967,0.194145,1286
Стајање,0.427319,0.425764,0.426540,1374
Лежање,0.382338,0.550817,0.451369,1407
Тачност,0.534956,0.534956,0.534956,1
Макро просјек,0.541546,0.547738,0.538076,7352
Пондерисани просјек,0.525921,0.534956,0.523355,7352


Тачност процијењених тестних података: 0.568375


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.662903,0.828629,0.736559,496
Пењање уз степенице,0.869668,0.779193,0.821948,471
Силазак низ степенице,0.626437,0.519048,0.567708,420
Сједење,0.320879,0.297352,0.308668,491
Стајање,0.485000,0.364662,0.416309,532
Лежање,0.482906,0.631285,0.547215,537
Тачност,0.568375,0.568375,0.568375,1
Макро просјек,0.574632,0.570028,0.566401,2947
Пондерисани просјек,0.568853,0.568375,0.562537,2947


Објашњена варијанса сваке од главних компоненти: [284.882377  36.917616]
Постотак објашњене варијансе сваке од главних компоненти: [0.507812 0.065807]
Укупни проценат кумулативне објашњене варијансе  57.3619
Kraj izvršenja:  2025-04-20 23:21:25.589395
Trajanje izvršenja u sekundama  77.870238


In [21]:
log_reg_sa_SVD(X_train,y_train,X_test,y_test,broj_komponenti=1)

Početak izvršenja:  2025-04-20 23:21:25.600955
Матрица добијена примјеном SVD на тестне податке 
 [[-10.945119]
 [-14.113888]
 [-15.324244]
 ...
 [ 13.045278]
 [  9.080038]
 [ 10.361516]]
Mатрица десно сингуларних вектора V^T 
 [[ 2.8000e-05 -2.4730e-03 -1.5270e-03  5.6213e-02  5.6788e-02  5.4363e-02
   5.5969e-02  5.6264e-02  5.3886e-02  5.5892e-02  5.4737e-02  5.3244e-02
  -5.6290e-02 -5.5753e-02 -5.3452e-02  5.7510e-02  5.1202e-02  5.1901e-02
   4.6770e-02  5.4928e-02  5.4387e-02  5.2070e-02  4.8122e-02  5.0300e-02
   4.7775e-02 -4.0788e-02  3.3422e-02 -2.0071e-02  1.5720e-02 -3.2300e-02
   3.1810e-02 -9.2440e-03 -7.8400e-04 -2.9816e-02  3.1502e-02 -1.4680e-02
   7.8040e-03 -2.3167e-02 -4.3140e-03  1.0851e-02  2.3810e-02 -3.0993e-02
  -2.7680e-02  9.1120e-03  9.8920e-03  1.1699e-02  9.1410e-03  9.8980e-03
   1.1572e-02  2.4526e-02 -3.0393e-02 -2.6963e-02  2.3144e-02 -3.1450e-02
  -2.8389e-02 -7.6580e-03  2.3551e-02 -2.0082e-02 -1.9833e-02  9.2750e-03
   9.8630e-03  1.1097e-02  2.393

[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.3s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.3s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.1s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.5s
[CV] END C=0.01, class_weight=balanced, max_iter=2000, multi_class=multinomial, penalty=l2, solver=saga; total time=   0.2s
[CV] END C=0.01, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.01, class_weight=balanced, max_iter=2500, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[

[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; total time=   0.1s
[CV] END C=0.05, class_weight=balanced, max_iter=2800, multi_class=multinomial, penalty=l2, solver=newton-cg; tot

,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.399703,0.219413,0.283307,1226
Пењање уз степенице,0.471467,0.646785,0.545383,1073
Силазак низ степенице,0.547268,0.639959,0.589995,986
Сједење,0.324742,0.048989,0.085135,1286
Стајање,0.434816,0.412664,0.423450,1374
Лежање,0.388498,0.705757,0.501136,1407
Тачност,0.437568,0.437568,0.437568,0
Макро просјек,0.427749,0.445594,0.404734,7352
Пондерисани просјек,0.421273,0.437568,0.395902,7352


Тачност процијењених тестних података: 0.444181


,Прецизност,Одзив,f1-score,Број узорака у класи
Пјешачење,0.358209,0.193548,0.251309,496
Пењање уз степенице,0.498437,0.677282,0.574257,471
Силазак низ степенице,0.524896,0.602381,0.560976,420
Сједење,0.377358,0.040733,0.073529,491
Стајање,0.485175,0.338346,0.398671,532
Лежање,0.389232,0.821229,0.528144,537
Тачност,0.444181,0.444181,0.444181,0
Макро просјек,0.438885,0.445587,0.397814,2947
Пондерисани просјек,0.436140,0.444181,0.394484,2947


Објашњена варијанса сваке од главних компоненти: [284.882377]
Постотак објашњене варијансе сваке од главних компоненти: [0.507812]
Укупни проценат кумулативне објашњене варијансе  50.781200000000005
Kraj izvršenja:  2025-04-20 23:22:01.998428
Trajanje izvršenja u sekundama  36.397473
